# Cartographie des Précipitations du 9 Octobre 2024


L'objectif de cette analyse est de préparer les données pour une carte du volume de précipitation par poste pour le 9 octobre 2024. Le fichier de données utilisé provient du site data.gouv.fr et contient des données météorologiques journalières relevées à partir de différents postes météorologiques. L'URL source est : https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_95_latest-2024-2025_RR-T-Vent.csv.gz. Ce fichier est particulièrement utile pour analyser les tendances climatiques et cartographier les données météorologiques.


Les données incluent diverses mesures telles que les précipitations, les températures minimales et maximales, la vitesse du vent, etc. Pour cette analyse, nous nous concentrerons sur les précipitations relevées le 9 octobre 2024. Nous utiliserons les champs `NUM_POSTE`, `LAT`, `LON`, et `RR` pour créer une carte représentant le volume de précipitation par poste.


## Méthodologie


La méthodologie utilisée pour cette analyse comporte plusieurs étapes. Tout d'abord, nous avons sélectionné les données pertinentes en utilisant une requête SQL pour filtrer les données du 9 octobre 2024 et ne garder que celles de bonne qualité (`QRR = '1'`). Les données ont ensuite été traitées et visualisées à l'aide de bibliothèques Python telles que `duckdb`, `pandas`, `plotly`, et `geopandas`. La visualisation finale est une carte interactive représentant le volume de précipitation par poste.


La carte est créée en utilisant `plotly.express` et `geopandas` pour superposer les données de précipitation sur une carte du département 95. Les précipitations sont représentées par des cercles dont la taille et la couleur varient en fonction du volume de précipitation. Cette visualisation permet une compréhension facile et intuitive de la distribution des précipitations dans la région.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_95_latest-2024-2025_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "NUM_POSTE",
  CAST("RR" AS DOUBLE) AS precipitation_volume,
  "LAT",
  "LON"
FROM 
  loaded_dataset
WHERE 
  CAST("AAAAMMJJ" AS VARCHAR) = '20241009'
  AND "QRR" = '1'  -- On ne garde que les données de précipitation de bonne qualité
ORDER BY 
  "NUM_POSTE" """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 6 lignes


,NUM_POSTE,precipitation_volume,LAT,LON
0,95078001,48.8,49.090333,2.028500
1,95088001,69.6,48.967333,2.427667
2,95492001,71.2,49.040333,2.406667
3,95527001,69.8,49.015167,2.534333
4,95580001,71.0,49.085667,2.550667


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui permet de créer des visualisations interactives et personnalisables. Cette technologie est adaptée pour représenter des données géospatiales, comme dans ce cas, où des données de précipitation sont superposées à une carte. Plotly offre une grande flexibilité pour créer des cartes interactives avec des informations détaillées.

In [4]:
import pandas as pd
import duckdb as ddb
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd

df['LAT'] = df['LAT'].astype(float)
df['LON'] = df['LON'].astype(float)

url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'
deps = gpd.read_file(url)
deps = deps.to_crs(4326)
dep95 = deps[deps['code'] == '95']

fig = px.choropleth_mapbox(
    dep95,
    geojson=dep95.geometry,
    locations=dep95.index,
    color_discrete_sequence=['#f0f0f0'],
    opacity=0.4,
    center=dict(lat=49.0, lon=2.15),
    zoom=9.2,
    mapbox_style='carto-positron',
)

fig.add_trace(
    go.Scattermapbox(
        lat=df['LAT'],
        lon=df['LON'],
        mode='markers',
        marker=dict(
            size=df['precipitation_volume'] * 0.2,
            color=df['precipitation_volume'],
            colorscale='Blues',
            showscale=True,
            colorbar=dict(
                title="Précip.<br>(mm)",
                thickness=15,
                x=0.98,
                y=0.5,
                len=0.7
            )
        ),
        text=[f"Poste {n}<br>{v} mm" for n, v in zip(df['NUM_POSTE'], df['precipitation_volume'])],
        hovertemplate="%{text}<extra></extra>",
        showlegend=False,
    )
)

fig.update_layout(
    title=dict(
        text="Volume de précipitation par poste le 9 octobre 2024",
        x=0.5,
        font=dict(size=16, family='DejaVu Sans')
    ),
    annotations=[
        dict(
            text="Source : données station météo IDF le 9/10/24",
            x=1,
            y=-0.02,
            showarrow=False,
            font=dict(size=11, color="gray"),
            xanchor='right'
        )
    ],
    margin=dict(l=0, r=0, t=50, b=0),
    height=700,
)

dataviz = fig
dataviz

C:\Users\ami\AppData\Local\Temp\ipykernel_15656\2626121031.py:16: DeprecationWarning: *choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.choropleth_mapbox(


C:\Users\ami\AppData\Local\Temp\ipykernel_15656\2626121031.py:28: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250804_065809.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250804_065809.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250804_065809.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250804_065809.png


--> Image Plotly sauvegardée avec succès.
